In [20]:
# Hospital
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
#from api_keys import g_key

#Matt's code
import sys
path = "/Users/meria/Desktop"
sys.path.insert(0, path)
from apifile import API_KEY
gkey= API_KEY["api_google"]

In [21]:
# Read the  data lat and lon from cities
cities = pd.read_csv("GeoIDs - City.csv")
cities.head()
#Create a new column to storage the Health Score for each city
#Note: The Health Score of each city is defined as:
#      average of the ratings available on google api  for places target_type = "hospital"
#      around 3000 meters of each city location

cities["Health score"] = ""
cities.head()

,cityid,cityname,stateabbrev,statename,statefips,lat,lon,city_pop2019,Health score
0,1,Los Angeles,CA,California,6,34.05,-118.24,10039107,
1,2,New York City,NY,New York,36,40.71,-74.01,8336817,
2,3,Chicago,IL,Illinois,17,41.88,-87.63,5150233,
3,4,Houston,TX,Texas,48,29.76,-95.37,4713325,
4,5,Phoenix,AZ,Arizona,4,33.45,-112.07,4485414,


In [22]:
#Gmaps
gmaps.configure(api_key = gkey)
#location
location = cities[["lat","lon"]]

In [23]:
#geocordinates
target_radius = 3000 #meters
target_type = "hospital"
ratings = "rating"

#set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "rating": ratings,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [24]:
#Empty counter to iterate over the cities in the for loop
k=0

for col,row in cities[:].iterrows():
    #Get  the Latitude and Longitude.
    lat =row["lat"]
    lng = row["lon"]  
    params["location"] = f"{lat},{lng}"

    # Request to get the Hospital Rating using url and parameters.
    response = requests.get(base_url, params=params)
    city = response.json()
    
    # empty list to get the rating values around the city in the specific radius 
    rating = []
    no_rating = 0
    
    #looking for hospital rating and storing the values in a list when a value is available
    for i in range (0,len(city["results"])+1):
        try:
            rating.append(city["results"][i]["rating"])

        except:
            no_rating = no_rating +1
    
    #print(rating)
    
    #calculate the average of the rating values found
    avg_rating = sum(rating)/len(rating)
    #print(avg_rating)
    
    #insert the rating average in an specific location (row = k) at the cities Data Frame, Health score column
    cities["Health score"].iloc[(k)]=avg_rating
    
    #counter that will go to the next city in the DataFrame
    k=k+1

C:\Users\meria\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [25]:
cities

,cityid,cityname,stateabbrev,statename,statefips,lat,lon,city_pop2019,Health score
0,1,Los Angeles,CA,California,6,34.05,-118.24,10039107,4.23333
1,2,New York City,NY,New York,36,40.71,-74.01,8336817,3.78421
2,3,Chicago,IL,Illinois,17,41.88,-87.63,5150233,4.07778
3,4,Houston,TX,Texas,48,29.76,-95.37,4713325,4.01818
4,5,Phoenix,AZ,Arizona,4,33.45,-112.07,4485414,4.03077
5,6,San Diego,CA,California,6,32.72,-117.16,3338330,3.95385
6,7,Dallas,TX,Texas,48,32.78,-96.80,2635516,3.375
7,8,Las Vegas,NV,Nevada,32,36.17,-115.14,2266715,3.26667
8,9,Seattle,WA,Washington,53,47.61,-122.33,2252782,3.46875
9,10,Fort Worth,TX,Texas,48,32.76,-97.33,2102515,4.1


In [26]:
#cities["RANK"]=
#cities=cities.set_index("cityname")

# ranked 1 (worst) to 5(best)
x = cities["Health score"].rank(ascending=True)

# population. If user does not want to move to city with high pop, ascending = false calls high pop worse.
y = cities["city_pop2019"].rank(ascending=False)

# able to add ranks for overall ranking for a summary score and add to dataframe.
cities['temp'] =  x+y

#able to sort the overall ranked dataframe.
cities.sort_values(by =["temp"],ascending=False)

,cityid,cityname,stateabbrev,statename,statefips,lat,lon,city_pop2019,Health score,temp
50,51,Virginia Beach,VA,Virginia,51,36.85,-75.98,449974,5,105.0
43,44,New Orleans,LA,Louisiana,22,29.95,-90.07,390144,4.34444,100.0
48,49,Boise,ID,Idaho,16,43.62,-116.20,481587,4.40833,99.0
23,24,Portland,OR,Oregon,41,45.52,-122.68,812855,4.6,88.0
29,30,Nashville,TN,Tennessee,47,36.16,-86.78,694144,4.2,86.5
39,40,Omaha,NE,Nebraska,31,41.25,-96.00,571327,4.02857,82.0
24,25,Boston,MA,Massachusetts,25,42.36,-71.06,803907,4.23333,81.5
51,52,Colorado Springs,CO,Colorado,8,38.83,-104.82,720403,4.16364,81.0
46,47,Bakersfield,CA,California,6,35.37,-119.02,900202,4.2,75.5
38,39,Raleigh,NC,North Carolina,37,35.78,-78.64,1111761,4.48,74.0


In [45]:
#Exporting the DataFrame with the rank and scores in a csv file and reading the DataFrame
cities_health_df = cities  

cities_health_df.to_csv("cities_health_df.csv")
cities_health_df.head() 

,cityid,stateabbrev,statename,statefips,lat,lon,city_pop2019,Health score,RANK,temp
cityname,,,,,,,,,,
Los Angeles,1,CA,California,6,34.05,-118.24,10039107,4.23333,43.5,44.5
New York City,2,NY,New York,36,40.71,-74.01,8336817,3.78421,21.0,23.0
Chicago,3,IL,Illinois,17,41.88,-87.63,5150233,4.07778,36.0,39.0
Houston,4,TX,Texas,48,29.76,-95.37,4713325,4.00909,32.0,36.0
Phoenix,5,AZ,Arizona,4,33.45,-112.07,4485414,4.03077,34.0,39.0
